In [1]:
import cv2
import numpy as np
import copy
import math
import time
import os
import pyautogui as py

In [2]:
def nothing(x):
    pass
image_x, image_y = 64,64
s=''
cap_region_x_begin=0.5  # start point/total width
cap_region_y_end=0.8  # start point/total width
threshold = 60  #  BINARY threshold
blurValue = 41  # GaussianBlur parameter
bgSubThreshold = 50
learningRate = 0
val=50.0
image_x, image_y = 64, 64

# variables
isBgCaptured = 0   # bool, whether the background captured
triggerSwitch = False  # if true, keyborad simulator works



In [3]:
import pickle
clf = pickle.load(open('modelhand3.sav', 'rb'))

In [11]:
t=[[]]
ma=0
isBgCaptured=0
cou=0
cv2.namedWindow("Trackbars")

cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 50, 255, nothing)

def printThreshold(thr):
    print("! Changed threshold to "+str(thr))


def removeBG(frame):
    fgmask = bgModel.apply(frame,learningRate=learningRate)
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    # res = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    return res

camera = cv2.VideoCapture(0)
camera.set(10,200)
cv2.namedWindow('threshold')
cv2.createTrackbar('trh1', 'threshold', threshold, 100, printThreshold)
while camera.isOpened():
    k = cv2.waitKey(1)
    lower=np.array([90,90,90])
    upper=np.array([240,240,240])
    ret, frame = camera.read()
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    threshold = cv2.getTrackbarPos('trh1', 'threshold')
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # smoothing filter
    frame = cv2.flip(frame, 1)  # flip the frame horizontally
    cv2.rectangle(frame, (int(cap_region_x_begin * frame.shape[1]), 0),
                 (frame.shape[1], int(cap_region_y_end * frame.shape[0])), (255, 0, 0), 2)
    cv2.imshow('original', frame)


    if isBgCaptured == 1: 
        cou+=1
        # this part wont run until background captured
        mas = cv2.inRange(frame, lower, upper)
        #img = cv2.bitwise_and(frame, frame, mask=mas)
        img = removeBG(frame)
        img = img[0:int(cap_region_y_end * frame.shape[0]),
                    int(cap_region_x_begin * frame.shape[1]):frame.shape[1]]  # clip the ROI

        # for i in range(img.shape[0]):
        #   for j in range(img.shape[1]):
        #     if img[i][j][0]<140 or img[i][j][0]>180:
        #       img[i][j][0]=0
        #       img[i][j][1]=0
        #       img[i][j][2]=0
        l_h = cv2.getTrackbarPos("L - H", "Trackbars")
        l_s = cv2.getTrackbarPos("L - S", "Trackbars")
        l_v = cv2.getTrackbarPos("L - V", "Trackbars")
        u_h = cv2.getTrackbarPos("U - H", "Trackbars")
        u_s = cv2.getTrackbarPos("U - S", "Trackbars")
        u_v = cv2.getTrackbarPos("U - V", "Trackbars")

        

        lower_blue = np.array([l_h, l_s, l_v])
        upper_blue = np.array([u_h, u_s, u_v])
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        #mask = cv2.inRange(hsv, lower_blue, upper_blue)
        cv2.imshow("m", hsv)
        cv2.imshow('mask', img)

        # convert the image into binary image
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
        cv2.imshow('blur', blur)
        #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
        ret3,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        kernel = np.ones((5,5),np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        kernel = np.ones((5,5),np.uint8)
        dilation = cv2.dilate(opening,kernel,iterations = 1)
        im2, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            a = cv2.contourArea(cnt)
            if a>ma:
                c=cnt;
        x,y,w,h = cv2.boundingRect(c)
        imcrop = thresh[y:y+h, x:x+w]
        cv2.imshow('ori', thresh)
        cv2.imshow('final', dilation)
        cv2.imshow('final', imcrop)
        img = cv2.resize(imcrop, (image_x, image_y), interpolation = cv2.INTER_AREA)
        if cou%150 == 0:
            for l in range(64):
                for j in range(64):
                    t[0].append(int(img[l][j]/255))
            predicted = clf.predict(t)
            if predicted==0:
                #py.FAILSAFE = False
                #py.typewrite([' '])
                print(0)
                break
            elif predicted == 1:
                #py.FAILSAFE = False
                #py.typewrite(['f'])
                print(1)
                break
            else:
                print(predicted)
                break
    if k == 27:  # press ESC to exit
        break
    elif k == ord('b'):  # press 'b' to capture the background
        bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
        isBgCaptured = 1
        print( '!!!Background Captured!!!')
    elif k == ord('r'):  # press 'r' to reset the background
        bgModel = None
        triggerSwitch = False
        isBgCaptured = 0
        print ('!!!Reset BackGround!!!')
    elif k == ord('n'):
        triggerSwitch = True
    elif k== ord('x'):
        for l in range(64):
            for j in range(64):
                t[0].append(int(img[l][j]/255))
        predicted = clf.predict(t)
        t=[[]]
        print(predicted)
print ('!!!Trigger On!!!')
camera.release()
cv2.destroyAllWindows()

!!!Background Captured!!!
[4]
!!!Trigger On!!!


In [40]:
imgYCC = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)

In [41]:
cv2.imshow('bvekj',imgYCC)
cv2.waitKey(0)
cv2.destroyAllWindows()